# BERT Fine-tuning for RAID Dataset
Full BERT model (bert-base-uncased) - slower but potentially more accurate

In [ ]:
# Install dependencies
!pip install transformers torch datasets scikit-learn -q

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Imports and Configuration
import os
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
import joblib

from transformers import (
    BertTokenizer, BertForSequenceClassification,
    Trainer, TrainingArguments, EarlyStoppingCallback
)
from datasets import Dataset


# CONFIGURATION
TRAIN_PATH = "/content/drive/MyDrive/train_none.csv"
MODEL_DIR = "/content/drive/MyDrive/"

MODELS = ["cohere-chat", "gpt4", "mistral-chat", "mpt-chat", "llama-chat"]
RANDOM_STATE = 5

# BERT Configuration (Full BERT)
BERT_MODEL_NAME = "bert-base-uncased"
BERT_MAX_LENGTH = 256
BERT_BATCH_SIZE = 16  # Smaller batch for full BERT (uses more memory)
BERT_EPOCHS = 3
BERT_LEARNING_RATE = 2e-5

os.makedirs(MODEL_DIR, exist_ok=True)

print("Setup complete!")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

Setup complete!
CUDA available: True
GPU: Tesla T4
GPU Memory: 15.8 GB


In [ ]:
# Load and prepare data
print(f"Loading dataset from: {TRAIN_PATH}")
full_df = pd.read_csv(TRAIN_PATH)
print(f"Loaded {len(full_df)} rows")

# Split into train/test
train_df, test_df = train_test_split(
    full_df,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=full_df["model"]
)

# Filter to specific models
train_df = train_df[train_df["model"].isin(MODELS)]
test_df = test_df[test_df["model"].isin(MODELS)]

train_texts = train_df["generation"].astype(str)
train_labels = train_df["model"]
test_texts = test_df["generation"].astype(str)
test_labels = test_df["model"]

print(f"Train size: {len(train_texts)}")
print(f"Test size: {len(test_texts)}")
print(f"\nLabel distribution:\n{train_labels.value_counts()}")

# Label encoding
le = LabelEncoder()
y_train = le.fit_transform(train_labels)
y_test = le.transform(test_labels)

print(f"\nClasses: {list(le.classes_)}")

# Save label encoder
joblib.dump(le, os.path.join(MODEL_DIR, "label_encoder.pkl"))

# Save test data for later comparison
joblib.dump({
    'test_texts': test_texts,
    'y_test': y_test,
    'test_labels': test_labels,
    'test_df': test_df
}, os.path.join(MODEL_DIR, "test_data.pkl"))

Loading dataset from: /content/drive/MyDrive/train_none.csv
Loaded 467985 rows
Train size: 171149
Test size: 42787

Label distribution:
model
mpt-chat        42787
mistral-chat    42787
llama-chat      42787
gpt4            21394
cohere-chat     21394
Name: count, dtype: int64

Classes: ['cohere-chat', 'gpt4', 'llama-chat', 'mistral-chat', 'mpt-chat']


['/content/drive/MyDrive/test_data.pkl']

In [ ]:
# Fine-tune BERT
def prepare_dataset(texts, labels, tokenizer, max_length):
    encodings = tokenizer(
        texts.tolist(),
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors='pt'
    )
    return Dataset.from_dict({
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': labels
    })

bert_model_path = os.path.join(MODEL_DIR, "bert_model")

print("Fine-tuning BERT (Full)...")
print(f"  Model: {BERT_MODEL_NAME}")
print(f"  Max Length: {BERT_MAX_LENGTH}")
print(f"  Batch Size: {BERT_BATCH_SIZE}")
print(f"  Epochs: {BERT_EPOCHS}")

tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(
    BERT_MODEL_NAME,
    num_labels=len(le.classes_)
)

# Prepare datasets
print("\nTokenizing texts...")
train_dataset = prepare_dataset(train_texts, y_train, tokenizer, BERT_MAX_LENGTH)
eval_dataset = prepare_dataset(test_texts, y_test, tokenizer, BERT_MAX_LENGTH)

# Training arguments
training_args = TrainingArguments(
    output_dir=bert_model_path,
    num_train_epochs=BERT_EPOCHS,
    per_device_train_batch_size=BERT_BATCH_SIZE,
    per_device_eval_batch_size=BERT_BATCH_SIZE,
    learning_rate=BERT_LEARNING_RATE,
    weight_decay=0.01,
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2,
    warmup_steps=500,
    fp16=torch.cuda.is_available(),
    gradient_accumulation_steps=2,  # Helps with memory
    report_to="none",
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='macro')
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# Train
print("\nStarting training...")
trainer.train()

# Save model
model.save_pretrained(bert_model_path)
tokenizer.save_pretrained(bert_model_path)
print(f"\n✅ Model saved to: {bert_model_path}")

Fine-tuning BERT (Full)...
  Model: bert-base-uncased
  Max Length: 256
  Batch Size: 16
  Epochs: 3


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Tokenizing texts...

Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.366900,0.386287,0.856031,0.857932
2,0.227700,0.317190,0.885549,0.890102
3,0.130200,0.353409,0.895038,0.898159



✅ Model saved to: /content/drive/MyDrive/bert_model


In [ ]:
# Evaluate and save predictions
print("\nEvaluating model...")
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

bert_preds = []
bert_probs = []

for i in range(0, len(test_texts), BERT_BATCH_SIZE):
    batch_texts = test_texts.iloc[i:i+BERT_BATCH_SIZE].tolist()

    encodings = tokenizer(
        batch_texts,
        truncation=True,
        padding=True,
        max_length=BERT_MAX_LENGTH,
        return_tensors='pt'
    )
    encodings = {k: v.to(device) for k, v in encodings.items()}

    with torch.no_grad():
        outputs = model(**encodings)
        probs = torch.softmax(outputs.logits, dim=1)
        preds = torch.argmax(outputs.logits, dim=1)

    bert_preds.extend(preds.cpu().numpy())
    bert_probs.extend(probs.cpu().numpy())

bert_preds = np.array(bert_preds)
bert_probs = np.array(bert_probs)

# Save predictions
joblib.dump({
    'predictions': bert_preds,
    'probabilities': bert_probs
}, os.path.join(MODEL_DIR, "bert_full_predictions.pkl"))

print(f"\n{'='*50}")
print("BERT (Full) RESULTS")
print(f"{'='*50}")
print(f"Accuracy: {accuracy_score(y_test, bert_preds):.4f}")
print(f"Macro F1: {f1_score(y_test, bert_preds, average='macro'):.4f}")
print(f"\n✅ Predictions saved to: {MODEL_DIR}/bert_full_predictions.pkl")


Evaluating model...

BERT (Full) RESULTS
Accuracy: 0.8950
Macro F1: 0.8982

✅ Predictions saved to: /content/drive/MyDrive//bert_full_predictions.pkl
